In [1]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import datetime, traceback, requests, io, os, re, shutil, zipfile
import traceback
import gtfstk as gt
import tkinter as tk
from tkinter import filedialog
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FormatStrFormatter
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
def readOnlineFeed(url):
    r=requests.get(url)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('x')
    feed=gt.read_gtfs('x',dist_units='mi')
    shutil.rmtree('x')
    return feed
gtfs=readOnlineFeed(r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip")

In [3]:
route_names=gtfs.routes['route_long_name']
route_namesArray=route_names.unique()
route_names={i:route_namesArray[i] for i in range(len(route_names.unique()))}
route_names.update([((len(route_names)+1),'All')])
route_names=route_names.values()

In [4]:
def getRouteStops(route_long_name):
    feed=gtfs.stop_times.merge(gtfs.trips).merge(gtfs.routes).merge(gtfs.stops)
    route=feed.loc[feed['route_long_name']==route_long_name]
    route.sort_values(['trip_id','direction_id','stop_sequence'],inplace=True)
    route=route.groupby(['stop_id'],as_index=False,sort=False).first()
    routeStops=route['stop_name']
    routeStops={i:routeStops[i] for i in range(len(routeStops))}
    routeStops.update([((len(routeStops)+1),'All')])
    routeStops=routeStops.values()
    return routeStops

In [5]:
def button_otp_clicked():
    root.otp=filedialog.askopenfilename(initialdir="/",
                                        title="Select OTP Detail Report",
                                        defaultextension='.csv',
                                        filetypes=(("csv files","*.csv"),("all files","*.*")))
def button_save_clicked():
    root.save_file=filedialog.asksaveasfilename(initialdir = "/",
                                                title = "Select file",
                                                defaultextension='.xlsx',
                                                filetypes = (("xlsx files","*.xlsx"),("all files","*.*")))

def close_window():
    root.route=route.get()
    root.stop=stop.get()
    root.destroy()

root=tk.Tk()
root.title('On Time Performance Analytics')

route=tk.StringVar(root)
route.set('All')

stop=tk.StringVar(root)
stop.set('All')

label_otp=tk.Label(root,text="Select OTP Detail Report").grid(column=0,row=0)
button_gtfs=tk.Button(root,text="Browse",command=button_otp_clicked).grid(column=1,row=0)

label_out=tk.Label(root,text="Save Report").grid(column=0,row=1)
button_out=tk.Button(root,text="Browse",command=button_save_clicked).grid(column=1,row=1)

route_label=tk.Label(root,text="Pick a Route").grid(column=0,row=2)
route_choice=tk.OptionMenu(root,route,*route_names).grid(column=1,row=2)

button_run=tk.Button(root,text='Run',width=25,command=close_window)
button_run.grid(column=0,row=3)
root.mainloop()

In [6]:
if root.route!='All':
    stops=getRouteStops(root.route)
stops

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


dict_values(['Boise Airport', 'Elder & Vista NEM', 'Elder @ Wells Fargo NM', 'Owyhee & Catalina SEC', 'Owyhee & Cherry SEC', 'Owyhee & Lemhi SEC', 'Nez Perce & Owyhee NWC', 'Nez Perce & Latah NEC', 'Nez Perce & Roosevelt NEC', 'Roosevelt & Overland SEC', 'Roosevelt & Kootenai NEC', 'Roosevelt & Cassia SEC', 'Roosevelt & Albion SEC', 'Roosevelt & Rose Hill SEC', 'Rose Hill & Cleveland SWC', 'Latah & Rose Hill NEC', 'Latah & Alpine SEC', 'Latah & Kipling SEC', 'Latah & Meadow SEC', 'Americana & Latah SEC', 'Americana & Ann Morrison NEC', 'Americana & Shoreline SEC', 'Shoreline & 14th SWM', '13th @ Post Office SWC', '13th & Grand SWC', 'Main & 11th SWC', 'Main Street Station', '13th & Grand NEC', '13th @ Post Office NWM', 'Shoreline & 14th SEC', 'Americana & Shoreline SWC', 'Americana & Albertson Park NWC', 'Latah & Cresent Rim SWM', 'Latah & Kipling SWM', 'Latah & Alpine SWC', 'Latah & Rose Hill NWC', 'Rose Hill & Cleveland NEC', 'Roosevelt & Albion NWC', 'Roosevelt & Cassia NWC', 'Roose

In [7]:
def close_window():
    second.stop=stop.get()
    second.destroy()
if root.route!='All':
    stops=getRouteStops(root.route)
    second=tk.Tk()
    second.title('Select A Stop')
    
    stop=tk.StringVar(second)
    stop.set('All')
    
    label_stops=tk.Label(second,text="Pick a Stop").grid(column=0,row=0)
    stop_choice=tk.OptionMenu(second,stop,*stops).grid(column=1,row=0)
    
    button_run=tk.Button(second,text='Run',width=25,command=close_window).grid(column=0,row=1)
    second.mainloop()

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
route_colors=gtfs.routes[['route_long_name','route_color']].sort_values('route_long_name')
route_colors=dict(zip(route_colors['route_long_name'].str.strip(),("#"+route_colors['route_color'])))

{'BSU Express': '#008c95',
 'Broadway': '#685bc7',
 'CWI Shuttle': '#71cc98',
 'Caldwell Blvd': '#63619a',
 'Caldwell Express': '#97d700',
 'Cole/Victory': '#51284f',
 'East Nampa': '#f8971d',
 'Emerald': '#8b0e04',
 'Fairview - Towne Square Mall': '#e93cac',
 'Fairview Ustick': '#e93cac',
 'Five Mile': '#009ca6',
 'Five Mile Chinden': '#009ca6',
 'Garden City': '#ba0c2f',
 'Happy Day to Towne Square Mall': '#486b4c',
 'Harris Ranch via Parkcenter': '#ffcd00',
 'Hill Road': '#fbb161',
 'Maple Grove': '#b6bd00',
 'Nampa/Meridian Express': '#aa8066',
 'Orchard': '#0072ce',
 'Overland': '#009639',
 'Roosevelt': '#05c3de',
 'State Street': '#001489',
 'VA/Hyde Park Loop': '#934d11',
 'Vista': '#ff8200',
 'Warm Springs': '#00a9ce'}

In [17]:
def cleanOTPData(OTP_data):
    df=pd.read_csv(OTP_data,parse_dates=['DepartTime','ScheduledStopTime','ArriveTime','TripDate','ScheduledTripStart','ScheduledTripEnd'])
    terminal=df.groupby('TripName')['StopOrder'].max().reset_index(name='TripTerminal')
    df=df.merge(terminal)
    df['End']=False
    df['Start']=False
    df['Terminal']=False
    df.loc[df.StopOrder==df.TripTerminal,'End']=True
    df.loc[df.StopOrder==1,'Start']=True
    df.loc[(df.End)|(df.Start),'Terminal']=True
    df.loc[df.End,'DepartTime']=df['ArriveTime']
    df.loc[df.DepartTime<df.ArriveTime,['DepartTime','ArriveTime']]=df.loc[df.DepartTime<df.ArriveTime,['ArriveTime','DepartTime']]
    df.dropna(axis=1,inplace=True,how='all')
    df.dropna(axis=0,inplace=True,subset=['DepartTime'])
    dropCols=['MinutesEarly','MinutesLate','Group1Type','Group1','EarlyStop','LateStop','EarlyThreshold','LateThreshold','AgencyDescription']
    df.drop(dropCols,axis=1,inplace=True)
    return df
cleanOTPData(root.otp)

,MasterRouteName,TripName,StopName,TripDate,ScheduledTripStart,ScheduledTripEnd,StopOrder,ScheduledStopTime,ArriveTime,DepartTime,TripTerminal,End,Start,Terminal
0,Broadway,2_05:35_Weekday_82661,Boise Airport,2019-07-01,2019-07-01 05:35:00,2019-07-01 06:06:00,1,2019-07-01 05:35:00,2019-07-01 05:24:53,2019-07-01 05:37:17,28,False,True,True
1,Broadway,2_05:35_Weekday_82661,Federal Way @ Fred Meyer SWC,2019-07-01,2019-07-01 05:35:00,2019-07-01 06:06:00,6,2019-07-01 05:43:00,2019-07-01 05:43:42,2019-07-01 05:43:50,28,False,False,False
2,Broadway,2_05:35_Weekday_82661,Boise & Law NWC,2019-07-01,2019-07-01 05:35:00,2019-07-01 06:06:00,12,2019-07-01 05:48:00,2019-07-01 05:48:58,2019-07-01 05:49:44,28,False,False,False
3,Broadway,2_05:35_Weekday_82661,Broadway & Beacon NEC,2019-07-01,2019-07-01 05:35:00,2019-07-01 06:06:00,22,2019-07-01 05:58:00,2019-07-01 05:58:24,2019-07-01 05:58:37,28,False,False,False
4,Broadway,2_05:35_Weekday_82661,Main Street Station,2019-07-01,2019-07-01 05:35:00,2019-07-01 06:06:00,28,2019-07-01 06:06:00,2019-07-01 06:18:19,2019-07-01 06:18:19,28,True,False,True
5,Broadway,2_05:35_Weekday_82661,Boise Airport,2019-07-02,2019-07-02 05:35:00,2019-07-02 06:06:00,1,2019-07-02 05:35:00,2019-07-02 05:27:35,2019-07-02 05:37:03,28,False,True,True
6,Broadway,2_05:35_Weekday_82661,Federal Way @ Fred Meyer SWC,2019-07-02,2019-07-02 05:35:00,2019-07-02 06:06:00,6,2019-07-02 05:43:00,2019-07-02 05:43:41,2019-07-02 05:43:49,28,False,False,False
7,Broadway,2_05:35_Weekday_82661,Boise & Law NWC,2019-07-02,2019-07-02 05:35:00,2019-07-02 06:06:00,12,2019-07-02 05:48:00,2019-07-02 05:49:25,2019-07-02 05:50:11,28,False,False,False
8,Broadway,2_05:35_Weekday_82661,Broadway & Beacon NEC,2019-07-02,2019-07-02 05:35:00,2019-07-02 06:06:00,22,2019-07-02 05:58:00,2019-07-02 05:58:08,2019-07-02 05:58:17,28,False,False,False
9,Broadway,2_05:35_Weekday_82661,Main Street Station,2019-07-02,2019-07-02 05:35:00,2019-07-02 06:06:00,28,2019-07-02 06:06:00,2019-07-02 06:16:42,2019-07-02 06:16:42,28,True,False,True


In [ ]:
OTP_data['RelDepart']=OTP_data['DepartTime']-OTP_data['ScheduledStopTime']
OTP_data['RelArrive']=OTP_data['ArriveTime']-OTP_data['ScheduledStopTime']

In [ ]:
OTP_data['RelDepNum']=(pd.to_numeric(OTP_data['RelDepart'],downcast='integer'))/60000000000
OTP_data['RelArrNum']=(pd.to_numeric(OTP_data['RelArrive'],downcast='integer'))/60000000000

In [ ]:
OTP_data['ArrStatus']='On Time'
OTP_data.loc[OTP_data.RelArrNum<-1,'ArrStatus']='Early'
OTP_data.loc[OTP_data.RelArrNum>5,'ArrStatus']='Late'
OTP_data.loc[np.logical_and(OTP_data.Terminal==True,OTP_data.RelArrNum<5),'ArrStatus']='On Time'

In [ ]:
OTP_data.head()

In [ ]:
OTP_data['DepStatus']='On Time'
OTP_data.loc[OTP_data.RelDepNum<-1,'DepStatus']='Early'
OTP_data.loc[OTP_data.RelDepNum>5,'DepStatus']='Late'

In [ ]:
OTP_data['date']=OTP_data['ScheduledStopTime'].dt.date
OTP_data['time']=OTP_data['ScheduledStopTime'].dt.time
OTP_data['hour']=[r.hour for r in OTP_data.time]
OTP_data['minutes']=[r.hour*60+r.minute for r in OTP_data.time]
OTP_data['ToDdec']=OTP_data['minutes']/60
OTP_data.sort_values(by='MasterRouteName',inplace=True)

In [ ]:
OTP_data['Hold']=False
OTP_data.loc[np.logical_and(OTP_data.ArrStatus=='Early',OTP_data.DepStatus=='On Time'),'Hold']='Early to On Time'
# OTP_data['Hold'][(OTP_data['ArrStatus']=='Early') & (OTP_data['DepStatus']=='On Time')]='Early to On Time'
OTP_data.loc[np.logical_and(OTP_data.ArrStatus=='Early',OTP_data.DepStatus=='Early'),'Hold']='Early to Early'
# OTP_data['Hold'][(OTP_data['ArrStatus']=='Early') & (OTP_data['DepStatus']=='Early')]='Early to Early'
OTP_data.loc[np.logical_and(OTP_data.RelArrNum<-1,OTP_data.DepStatus=='Late'),'Hold']='Early to Late'
# OTP_data['Hold'][(OTP_data['RelArrNum']<-1) & (OTP_data['DepStatus']=='Late')]='Early to Late'
holds=OTP_data[OTP_data.Hold!=False]

In [ ]:
f, ax = plt.subplots(figsize=(17,11))
ax=sns.stripplot(x=holds.MasterRouteName,y=holds.ToDdec,size=2,hue=holds.Hold,palette={'Early to On Time':'lightgreen','Early to Early':'blue','Early to Late':'darkred'})
plt.xticks(rotation=90)
plt.xlabel("Route")
plt.ylabel("Time of Day")
plt.title("Early Arrivals")
plt.tight_layout()

In [ ]:
route_colors

In [ ]:
f, ax = plt.subplots(figsize=(15,6))
plt.axhspan(-1,5,color='green',alpha=.5)
violin=sns.violinplot(x=OTP_data.MasterRouteName,y=OTP_data.RelArrNum,ax=ax,scale="width",linewidth=0,palette=route_colors)
plt.xticks(rotation=90)
plt.yticks(np.arange(-5, 25, step=5))
plt.ylim(-5,25)
plt.ylabel("Minutes from Schedule")
plt.xlabel("Route")
plt.title("Variance from Schedule - Arrivals")
plt.tight_layout()

In [ ]:
f, ax = plt.subplots(figsize=(15,6))
violin=sns.violinplot(x=OTP_data.MasterRouteName,y=OTP_data.RelDepNum,ax=ax,scale="width",linewidth=0,palette=route_colors)
plt.xticks(rotation=90)
plt.yticks(np.arange(-5, 25, step=5))
plt.ylim(-5,25)
plt.ylabel("Minutes from Schedule")
plt.xlabel("Route")
plt.title("Variance from Schedule - Departures")
plt.axhspan(-1,5,color='green',alpha=.5)
plt.tight_layout()

In [ ]:
routeOTP=OTP_data.groupby(['MasterRouteName','ArrStatus','DepStatus'])
routeOTP=pd.DataFrame(routeOTP.size(),columns=['Count'])
routeOTP['Perc']=routeOTP.groupby(level=0).apply(lambda x: 100*x/x.sum())
# with pd.option_context('display.max_rows',None):
#     print(routeOTP)

In [ ]:
def Quantiler(RouteName,Stop):
    Stop_Quantiles=pd.DataFrame({'Route':[],
                             'StopName':[],
                             'ScheduledStopTime':[],
                             'Min':[],
                             'TenPercentQuant':[],
                             'TwentyFivePercentQuant':[],
                             'FiftyPercentQuant':[],
                             'SeventyFivePercentQuant':[],
                             'NinetyPercentQuant':[],
                             'Max':[],
                                'Range':[],
                                'Mid80':[]})
    cols=['Route','StopName','ScheduledStopTime','Min','TenPercentQuant','TwentyFivePercentQuant','FiftyPercentQuant','SeventyFivePercentQuant','NinetyPercentQuant','Max','Range','Mid80']
    route=OTP_data.loc[OTP_data['MasterRouteName']==RouteName]
    route=route.sort_values(['ScheduledTripStart','StopOrder'])
    route['TravelTime']=route.DepartTime.diff()
    route.loc[route.ScheduledTripStart!=route.ScheduledTripStart.shift(1),'TravelTime']=None
    route['TravelDec']=(route.TravelTime.dt.total_seconds())/60
    route.loc[route.TravelDec<=0,'TravelDec']=None
    route.loc[route.TravelDec>100,'TravelDec']=None
    route_stop=route.loc[route['StopName']==Stop]
    route_stop.dropna(subset=['TravelDec'],inplace=True)
    for i in route_stop['time'].unique():
        time=route_stop.loc[route_stop['time']==i]
        Route=time['MasterRouteName'].iloc[0]
        StopName=time['StopName'].iloc[0]
        ScheduledStopTime=i
        Min=time['TravelDec'].min()
        TenPercentQuant=time['TravelDec'].quantile(q=.1)
        TwentyFivePercentQuant=time['TravelDec'].quantile(q=.25)
        FiftyPercentQuant=time['TravelDec'].quantile(q=.5)
        SeventyFivePercentQuant=time['TravelDec'].quantile(q=.75)
        NinetyPercentQuant=time['TravelDec'].quantile(q=.9)
        Max=time['TravelDec'].max()
        Range=Max-Min
        Mid80=NinetyPercentQuant-TenPercentQuant
        time_Quantiles=pd.DataFrame({'Route':[Route],
                                     'StopName':[StopName],
                                     'ScheduledStopTime':[ScheduledStopTime],
                                     'Min':[round(Min,1)],
                                     'TenPercentQuant':[round(TenPercentQuant,1)],
                                     'TwentyFivePercentQuant':[round(TwentyFivePercentQuant,1)],
                                     'FiftyPercentQuant':[round(FiftyPercentQuant,1)],
                                     'SeventyFivePercentQuant':[round(SeventyFivePercentQuant,1)],
                                     'NinetyPercentQuant':[round(NinetyPercentQuant,1)],
                                     'Max':[round(Max,1)],
                                     'Range':[round(Range,1)],
                                     'Mid80':[round(Mid80,1)]
                                    })
        Stop_Quantiles=Stop_Quantiles.append(time_Quantiles)
    Stop_Quantiles=Stop_Quantiles[cols]
    Stop_Quantiles.sort_values('ScheduledStopTime',inplace=True)
    return Stop_Quantiles

In [ ]:
def timePlotter(RouteName,Stop):
    route=OTP_data.loc[OTP_data['MasterRouteName']==RouteName]
    route=route.sort_values(['ScheduledTripStart','StopOrder'])
    route['TravelTime']=route.DepartTime.diff()
    route.loc[route.ScheduledTripStart!=route.ScheduledTripStart.shift(1),'TravelTime']=None
    route['TravelDec']=(route.TravelTime.dt.total_seconds())/60
    route.loc[route.TravelDec<=0,'TravelDec']=None
    route.loc[route.TravelDec>100,'TravelDec']=None
    route_stop=route.loc[route['StopName']==Stop]
    route_stop.dropna(subset=['TravelDec'],inplace=True)
    f, ax = plt.subplots(figsize=(15,6))
    ax=sns.violinplot(x=route_stop.time,y=route_stop.TravelDec,inner='quartile')
    ax.grid()
    ax.set_ylim(route_stop['TravelDec'].min(),route_stop['TravelDec'].max())
    ax.set_title(route_stop['MasterRouteName'].iloc[0]+': '+route_stop['StopName'].iloc[0])
    ax.set(xlabel=None,ylabel='Travel Time')
    plt.xticks(rotation='vertical')
    plt.tight_layout()
    return ax

In [ ]:
routeName='Emerald'
stopName='River & 13th NWC'

In [ ]:
timePlotter(routeName,stopName)

In [ ]:
Quantiler(routeName,stopName)